In [1]:
import random
import numpy as np
from scipy.optimize import linear_sum_assignment as hungarian
from editdistance import eval as editdist
from pdb import set_trace as t

In [2]:
# html = ['animal','cat','dog','bird','1','2','3','color','oops','red','4','pet','dog','3','fig']
# pdf = ['animal','cat','1','fig','dog','2','bird','3','color','ref','4','pet','whoops','dog','3']

In [43]:
"""
TODO: 
make parallel lists
scramble locally
make some duplicate words
replace a few words entirely
cut some words in half

""" 
import copy
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def make_lists(N, 
               scramble_pct=0, 
               scramble_rng=10, 
               duplicate_pct=0, 
               mismatch_pct=.05,
               partial_pct=0,
               offset=0):
    duplicate_num = 0
    mismatch_num = 0
    partial_num = 0
    uid = 0
    
    # make parallel lists
    html = [(i, ''.join(random.sample(alphabet, random.randint(3,10)))) for i in range(N)]
    pdf = copy.deepcopy(html)
    uid = N
    
    # make some duplicate words
    for i in range(N):
        if random.random() < duplicate_pct:
            duplicate_num += 2
            html[i] = (uid, html[random.randint(0, N-1)][1])
            pdf[i] = html[i]
            uid += 1
    
    # replace some words
    for i in range(N):
        if random.random() < mismatch_pct:
            mismatch_num += 1
            html[i] = (None, html[i][1])
            pdf[i] = (None, ''.join(random.sample(alphabet, random.randint(3,10))))
            uid += 1
    
    # cut some words in half
    for i in range(N):
        if random.random() < partial_pct:
            partial_num += 1
            if random.random() < 0.5:
                html[i] = (html[i][0], html[i][1][:random.randint(2, len(html[i]))])
            else:
                pdf[i] = (pdf[i][0], pdf[i][1][:random.randint(2, len(html[i]))])
                                   
    # scramble locally
    old_pdf = pdf
    pdf = [None] * N
    i = 0
    j = 0
    for i in range(N):
        if i < (N - scramble_rng) and random.random() < scramble_pct:
            jump = random.randint(1,scramble_rng)
            k = j + jump
            while pdf[k] is not None:
                k += 1
            pdf[k] = old_pdf[i]
        else:
            while pdf[j] is not None:
                j += 1
            pdf[j] = old_pdf[i]  

    # offset 
    for i in range(offset):
        pdf = [(None, ''.join(random.sample(alphabet, random.randint(3,10))))] + pdf
    
    duplicate_pct = float(duplicate_num)/N
    mismatch_pct = float(mismatch_num)/N
    partial_pct = float(partial_num)/N

    return (html, pdf, duplicate_pct, mismatch_pct, partial_pct)

In [44]:
%%time
from pprint import pprint

N = 100
(html, pdf, dup, mis, part) = make_lists(N)
print "Duplicate: %s" % dup
print "Mismatch:  %s" % mis
print "Partial:   %s" % part
print "Aligned:   %s" % (sum([html[i][0] == pdf[i][0] for i in range(N)]) / float(N))
# pprint(zip(html, pdf))
# for i in max(len(html), len(pdf)):
#     print "%s:%s" % (html[i][1] if i < len(html) else 'None', pdf[i][1] if i < len(pdf) else 'None')

Duplicate: 0.0
Mismatch:  0.05
Partial:   0.0
Aligned:   1.0
CPU times: user 1.92 ms, sys: 201 µs, total: 2.12 ms
Wall time: 2.21 ms


Idea: hungarian does a good job matching, but it's expensive as an n^3 algorithm. But we know that matches won't be found from the top of one to the bottom of the other, so only give a small portion at a time to match. Use a sliding window of 'small' size and find matches. Then start the next window (correcting for the difference in i and j of the last members of the previous window).

In [45]:
%%time

def calculate_offset(listA, listB, offsetN, offsetM):
    wordsA = zip(*listA[:offsetN])[1]
    wordsB = zip(*listB[:offsetM])[1]
    offsets = []
    for i in range(offsetN):
        try:
            offsets.append(wordsB.index(wordsA[i]) - i)
        except:
            pass
    # DEBUGGING:
    for i, offset in enumerate(offsets):
        print (listA[i], listB[i + offset])
    # DEBUGGING
    return int(np.median(offsets))

offsetN = 10
offsetM = 100
offset = calculate_offset(html, pdf, offsetN, offsetM)

print "Offset: %d" % int(offset)

((0, 'ztwfo'), (0, 'ztwfo'))
((1, 'dyborct'), (1, 'dyborct'))
((2, 'mthucbw'), (2, 'mthucbw'))
((3, 'arobnghel'), (3, 'arobnghel'))
((None, 'acw'), (None, 'gevb'))
((5, 'kmgxeszhyl'), (5, 'kmgxeszhyl'))
((6, 'mrnacdy'), (6, 'mrnacdy'))
((7, 'vnbeah'), (7, 'vnbeah'))
Offset: 0
CPU times: user 493 µs, sys: 374 µs, total: 867 µs
Wall time: 844 µs


In [47]:
# define costs
index_dist_cost = 1
edit_dist_cost = 50

N = 100
M = 120
W = 10
scale = 1
offset = 0
for i in range(N/W + 1):
    wordsA = html[max(i*W,0):min((i+1)*W,N)]
    wordsB = pdf[max(i*W - W*(scale-1) + offset, 0):min((i+1)*W + W*(scale-1) + offset, M)]
    pprint(zip(wordsA, wordsB))
    

# start_html = html[:10]
# start_pdf  = pdf[:10]

# NOTE: pre-generate offset matrix once
# NOTE: base matrix size on W, not len(html)

# # start with index distances 
# x = np.arange(len(start_html))
# y = np.arange(len(start_pdf))
# xx, yy = np.meshgrid(x,y)
# X = (abs(xx - yy) + yy * .01) * index_dist_cost

# # add edit distances
# for i, a in enumerate(start_html):
#     for j, b in enumerate(start_pdf):
#         X[i,j] += editdist(a[1], b[1]) * edit_dist_cost

[((0, 'ztwfo'), (0, 'ztwfo')),
 ((1, 'dyborct'), (1, 'dyborct')),
 ((2, 'mthucbw'), (2, 'mthucbw')),
 ((3, 'arobnghel'), (3, 'arobnghel')),
 ((None, 'acw'), (None, 'gevb')),
 ((5, 'kmgxeszhyl'), (5, 'kmgxeszhyl')),
 ((6, 'mrnacdy'), (6, 'mrnacdy')),
 ((7, 'vnbeah'), (7, 'vnbeah')),
 ((None, 'hykvlr'), (None, 'pegajqlokz')),
 ((9, 'ixp'), (9, 'ixp'))]
[((10, 'qnd'), (10, 'qnd')),
 ((11, 'vmpnct'), (11, 'vmpnct')),
 ((None, 'pctrnl'), (None, 'raqyuxv')),
 ((13, 'waucxiepr'), (13, 'waucxiepr')),
 ((14, 'qcivfg'), (14, 'qcivfg')),
 ((15, 'ykm'), (15, 'ykm')),
 ((16, 'buzyfhxi'), (16, 'buzyfhxi')),
 ((17, 'krtc'), (17, 'krtc')),
 ((18, 'usr'), (18, 'usr')),
 ((19, 'wtngsrzh'), (19, 'wtngsrzh'))]
[((20, 'dbhpgwu'), (20, 'dbhpgwu')),
 ((21, 'jozcehudg'), (21, 'jozcehudg')),
 ((22, 'xrfiwgqlm'), (22, 'xrfiwgqlm')),
 ((23, 'ckgrp'), (23, 'ckgrp')),
 ((24, 'xoqjuwrnbh'), (24, 'xoqjuwrnbh')),
 ((25, 'tjzlpxvwh'), (25, 'tjzlpxvwh')),
 ((None, 'onvy'), (None, 'xoiykbj')),
 ((27, 'xweir'), (27, 'xwe

In [ ]:
# %%time
# row_ind, col_ind = hungarian(X)

In [30]:
%%time
X = np.random.rand(10,100)
for _ in range(400):
    hungarian(X)

CPU times: user 166 ms, sys: 3.91 ms, total: 170 ms
Wall time: 208 ms


In [ ]:
new_html = [html[row_ind[i]] for i in range(N)]
new_pdf = [pdf[col_ind[i]] for i in range(N)]

# partial = sum([(new_html[i] != new_pdf[i] and
#               (new_html[i].startswith(new_pdf[i]) or
#                new_pdf[i].startswith(new_html[i]))) for i in range(N)])
# print "Duplicate: %s" % ?
# print "Partial:   %f" % partial/N
# print "Mismatch:  %f" % sum([new_html[i] != new_pdf[i] for i in range(N)]) - partial

print "Aligned:   %s" % sum([new_html[i][0] == new_pdf[i][0] for i in range(N)])

for i in xrange(len(row_ind)):
    print html[row_ind[i]], pdf[col_ind[i]]

In [ ]:
print sum([html[row_ind[i]] == pdf[col_ind[i]] for i in range(N)])